In [4]:
from astropy.io import fits
from astropy.io import ascii as asc
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
import pandas as pd
import os
import json
import copy
import shutil as sh
import time
import regex as re

In [9]:
def lines_lister(file_path):
    with open(file_path) as opened_file:
        lines_in_file = opened_file.readlines()
    return lines_in_file

def addreplace_param(file_lines, param_to_replace, new_str, add_str = False, split_condition = ': '):
    new_lines = file_lines.copy()
    indexer = 0
    for line_index in range(len(file_lines)):
        line = file_lines[line_index]
        if len(line) == 0:
            continue
        
        if (line.startswith(param_to_replace)) & (line.find('#') > 0):
            line_elements = line.split('#')[0].split(split_condition)
            old_str = line_elements[1].strip()
            if add_str == True:
                final_str = line_elements[0] + split_condition + old_str + str(new_str) + ' #' + line.split('#')[1]
            elif add_str == False:
                final_str = line_elements[0] + split_condition + str(new_str) + ' #' + line.split('#')[1]

            new_lines[line_index] = final_str
            indexer += 1
            
        elif line.startswith(param_to_replace):
            line_elements = line.split(split_condition)
            old_str = line_elements[1].strip()
            if add_str == True:
                final_str = line_elements[0] + split_condition + old_str + str(new_str) + '\n'
            elif add_str == False:
                final_str = line_elements[0] + split_condition + str(new_str) + '\n'
            
            new_lines[line_index] = final_str
            indexer += 1  

    if indexer <= 0:
        print('replacing the param has not worked')
    
    return new_lines

def write_file(new_file_dir, updated_line_list: list):
    with open(new_file_dir, 'w') as overwritten:
        overwritten.writelines(updated_line_list)

file_dir = '/Users/mokshahuja/Desktop/14_RD_pipeline/config_pipe/image.sex'

lines_to_use = lines_lister(file_dir)
newlines = addreplace_param(lines_to_use, 'DETECT_THRESH', new_str= 2.0, add_str= False, split_condition= '    ')
write_file(file_dir, newlines)



In [10]:
splitfile = os.path.splitext(file_dir)
print(splitfile)

('/Users/mokshahuja/Desktop/14_RD_pipeline/config_pipe/image', '.sex')


In [3]:
np.random.normal(scale = 0.3, size = 10)

array([-0.29361392,  0.0622676 ,  0.28513627, -0.17240359,  0.1977875 ,
        0.65671181, -0.16480312,  0.0502017 ,  0.14465899, -0.23694494])

In [2]:
data ={
        "20": {
            "False_Detections": 262,
            "Total_Inserted": 15,
            "Total_Detected": 11
        },
        "21": {
            "False_Detections": 264,
            "Total_Inserted": 15,
            "Total_Detected": 8
        },
        "22": {
            "False_Detections": 268,
            "Total_Inserted": 15,
            "Total_Detected": 10
        }
    }

In [2]:
data2 = {}

In [2]:
def convert_np(obj):
    if isinstance(obj, np.int64):
        return int(obj)

def column_dict_summer(dict_to_use, columns):
    final_dict = {}
    for column in columns:
        for each_item in dict_to_use:
            final_dict[column] = final_dict.get(column, 0) + dict_to_use[each_item][column]
    return final_dict


final_dict = column_dict_summer(data, ['False_Detections', 'Total_Inserted', 'Total_Detected'])
print(json.dumps(final_dict, indent=4, default= convert_np))      

KeyError: 'False_Detections'

In [1]:
import pandas as pd
import numpy as np

# Creating a random DataFrame with 3 rows and 3 columns
df1 = pd.DataFrame({
    'A': np.random.randint(1, 10, 3),
    'B': np.random.randint(10, 20, 3),
    'C': np.random.randint(20, 30, 3)
})

print("DataFrame 1:")
print(df1)

# Creating another random DataFrame with 5 rows and 3 columns
df2 = pd.DataFrame({
    'A': np.random.randint(1, 10, 5),
    'B': np.random.randint(10, 20, 5),
    'C': np.random.randint(20, 30, 5)
})

print("DataFrame 2:")
print(df2)



DataFrame 1:
   A   B   C
0  6  17  23
1  7  18  29
2  2  14  27
DataFrame 2:
   A   B   C
0  5  10  21
1  3  12  24
2  8  19  20
3  8  16  23
4  2  13  20


In [4]:
df_final = pd.concat([df1, df2])
print(df_final.reset_index())

   index  A   B   C
0      0  6  17  23
1      1  7  18  29
2      2  2  14  27
3      0  5  10  21
4      1  3  12  24
5      2  8  19  20
6      3  8  16  23
7      4  2  13  20


In [3]:
data = {
                    "20": {
                        "False_Detections": 13,
                        "Total_Inserted": 15,
                        "Total_Detected": 12,
                        "Completeness": 80.0,
                        "Reliability": 48.0
                    },
                    "21": {
                        "False_Detections": 17,
                        "Total_Inserted": 15,
                        "Total_Detected": 8,
                        "Completeness": 53.33,
                        "Reliability": 32.0
                    },
                    "22": {
                        "False_Detections": 12,
                        "Total_Inserted": 15,
                        "Total_Detected": 13,
                        "Completeness": 86.67,
                        "Reliability": 52.0
                    }
                }

In [5]:
def column_dict_summer(dict_to_use, column_ops_dict: dict):
    final_dict = {}
    for column in column_ops_dict:
        for each_item in dict_to_use:
            # print('LEN', (dict_to_use))
            if len(dict_to_use[each_item]) > 0:
                final_dict[column] = final_dict.get(column, 0) + dict_to_use[each_item][column]
                
        if column_ops_dict[column] == 'mean':
            final_dict[column] = round(final_dict[column]/len(dict_to_use), 2)         

    return final_dict

column_operations_dict = {
    'False_Detections': 'sum', 
    'Total_Inserted': 'sum', 
    'Total_Detected': 'sum',
    'Completeness': 'mean',
    'Reliability': 'mean'
}
final_dict = column_dict_summer(data, column_operations_dict)
print(final_dict)

{'False_Detections': 42, 'Total_Inserted': 45, 'Total_Detected': 33, 'Completeness': 73.33, 'Reliability': 44.0}


In [6]:
print('hello')

hello
